# go-OOP思想和方法

### 面向对象

* go也支持OOP,传统的类由成员函数和成员变量组成
* go语言中的方法则是一个一个和特殊类型关联的函数
* 一个面向对象的程序会用方法来表达其属性和对应的操作，这样使用这个对象的用户就不需要直接去操作对象，而是借助方法来做这些事情。

### method

带有接收者的函数，我们称为method, 这个接收者相当于类中的this,self.

method是附属在一个给定的类型上的，语法和函数的声明语法几乎一样，只是在func后面增加了一个receiver(也就是method所依从的主体)

In [ ]:
package main

import (
	"fmt"
	"math"
)

type Rectangle struct {
	width, height float64
}

type Circle struct {
	radius float64
}

func (r Rectangle) area() float64 {
	return r.width*r.height
}

func (c Circle) area() float64 {
	return c.radius * c.radius * math.Pi
}


func main() {
	r1 := Rectangle{12, 2}
	r2 := Rectangle{9, 4}
	c1 := Circle{10}
	c2 := Circle{25}

	fmt.Println("Area of r1 is: ", r1.area())
	fmt.Println("Area of r2 is: ", r2.area())
	fmt.Println("Area of c1 is: ", c1.area())
	fmt.Println("Area of c2 is: ", c2.area())
}

上面的代码func后里那个附加的参数，叫方法的接收器(receiver)，早期的面向对象语言留下的遗产将调用一个方法称为“向一个对象发送消息”。

Go语言中，没有像其它语言那样用this或self作为接收器；

Go语言中可以任意选择接收器的名字。

由于接收器的名字经常会被使用到，所以保持其在方法间传递时的一致性和简短性是不错的主意。

这里的建议是使用其类型的第一个字母，比如Point的首字母p。

在方法调用过程中，接收器参数一般会在方法名之前出现。

注意点:
* 虽然method的名字一模一样，但是如果接收者不一样，那么method就不一样
* method里面可以访问接收者的字段
* 调用method通过.访问，就像struct里面访问字段一样

说明: Receiver可以是**值传递**，还可以是**指针**,两者的差别在于: 
* 指针作为Receiver会对实例对象的内容发生操作.
* 普通类型作为Receiver仅仅是以副本作为操作对象,并不对原实例对象发生操作.

### method是否只能作用于struct？

当然不是，method可以定义在任何你自定义的类型、内置类型、struct等各种类型上面。

这里你是不是有点迷糊了，什么叫自定义类型? 自定义类型不就是struct嘛? 

不是这样的，struct只是自定义类型里面一种比较特殊的类型而已，还有其他自定义类型申明，可以通过如下这样的申明来实现。
`type typeName typeLiteral`

实际上只是一个定义了一个别名,有点类似于c中的typedef，


### 指针作为recevirer

In [ ]:
package main

import "fmt"

const(
	WHITE = iota
	BLACK
	BLUE
	RED
	YELLOW
)

type Color byte

type Box struct {
	width, height, depth float64
	color Color
}

type BoxList []Box //a slice of boxes

func (b Box) Volume() float64 {
	return b.width * b.height * b.depth
}

func (b *Box) SetColor(c Color) {
	b.color = c
}

func (bl BoxList) BiggestColor() Color {
	v := 0.00
	k := Color(WHITE)
	for _, b := range bl {
		if bv := b.Volume(); bv > v {
			v = bv
			k = b.color
		}
	}
	return k
}

func (bl BoxList) PaintItBlack() {
	for i := range bl {
		bl[i].SetColor(BLACK)
	}
}

func (c Color) String() string {
	strings := []string {"WHITE", "BLACK", "BLUE", "RED", "YELLOW"}
	return strings[c]
}

func main() {
	boxes := BoxList {
		Box{4, 4, 4, RED},
		Box{10, 10, 1, YELLOW},
		Box{1, 1, 20, BLACK},
		Box{10, 10, 1, BLUE},
		Box{10, 30, 1, WHITE},
		Box{20, 20, 20, YELLOW},
	}

	fmt.Printf("We have %d boxes in our set\n", len(boxes))
	fmt.Println("The volume of the first one is", boxes[0].Volume(), "cm³")
	fmt.Println("The color of the last one is",boxes[len(boxes)-1].color.String())
	fmt.Println("The biggest one is", boxes.BiggestColor().String())

	fmt.Println("Let's paint them all black")
	boxes.PaintItBlack()
	fmt.Println("The color of the second one is", boxes[1].color.String())

	fmt.Println("Obviously, now, the biggest one is", boxes.BiggestColor().String())
}

在`SetColor`函数里要读取到指针相应的值, 原本应该这样定义`*b.Color=c`,而不是`b.Color=c`, 但在go中却可以用`b.Color=c`这种方式呢?

答案是: 在Go里面这两种方式都是正确的，当用指针去访问相应字段时(虽然指针没有任何字段)，但是Go知道你要通过指针去获取这个值。

在`PaintItBlack`里面调用`SetColor`的时候是不是应该写成`(&bl[i]).SetColor(BLACK)`，因为SetColor的receiver是`*Box`，而不是Box?

答案是: 这两种方式都可以，因为Go知道receiver是指针，GO自动帮你转了。

也就是说：

如果一个method的receiver是`*T`,你可以在一个T类型的实例变量V上面调用这个method，而不需要`&V`去调用这个method

类似的

如果一个method的receiver是T，你可以在一个`*T`类型的变量P上面调用这个method，而不需要 `*P`去调用这个method

所以，不用担心你是调用的指针的method还是不是指针的method，Go知道你要做的一切.

### method继承

在Go中struct中的字段可以被继承,method也可以被继承,如果匿名字段实现了一个method，那么包含这个匿名字段的struct也能调用该method.
```
package main

import "fmt"

type Human struct {
	name string
	age int
	phone string
}

type Student struct {
	Human //匿名字段
	school string
}

type Employee struct {
	Human //匿名字段
	company string
}

//在human上面定义了一个method
func (h *Human) SayHi() {
	fmt.Printf("Hi, I am %s you can call me on %s\n", h.name, h.phone)
}

func main() {
	mark := Student{Human{"Mark", 25, "222-222-YYYY"}, "MIT"}
	sam := Employee{Human{"Sam", 45, "111-888-XXXX"}, "Golang Inc"}

	mark.SayHi()
	sam.SayHi()
}
```

### method重写

在上面的例子中，如果Employee想要实现自己的SayHi,怎么办？

答: 和匿名字段冲突一样的道理，在Employee上面定义一个method，重写了匿名字段的方法。请看下面的例子

```
package main

import "fmt"

type Human struct {
	name string
	age int
	phone string
}

type Student struct {
	Human //匿名字段
	school string
}

type Employee struct {
	Human //匿名字段
	company string
}

//Human定义method
func (h *Human) SayHi() {
	fmt.Printf("Hi, I am %s you can call me on %s\n", h.name, h.phone)
}

//Employee的method重写Human的method
func (e *Employee) SayHi() {
	fmt.Printf("Hi, I am %s, I work at %s. Call me on %s\n", e.name,
		e.company, e.phone) //Yes you can split into 2 lines here.
}

func main() {
	mark := Student{Human{"Mark", 25, "222-222-YYYY"}, "MIT"}
	sam := Employee{Human{"Sam", 45, "111-888-XXXX"}, "Golang Inc"}

	mark.SayHi()
	sam.SayHi()
}
```


### go中类成员函数声明

在函数声明时，在其名字之前放上一个变量，即是一个方法。这个附加的参数会将该函数附加到这种类型上，即相当于为这种类型定义了一个独占的方法。

### 方法调用

在方法调用过程中，接收器参数一般会在方法名之前出现。这和方法声明是一样的，都是接收器参数在方法名字之前。下面是例子：
```
p := Point{1, 2}
q := Point{4, 6}
fmt.Println(Distance(p, q)) // "5", function call
fmt.Println(p.Distance(q))  // "5", method call
```

上面的两个函数调用都是Distance，但是却没有发生冲突。第一个Distance的调用实际上用的是包级别的函数geometry.Distance，而第二个则是使用刚刚声明的Point，调用的是Point类下声明的Point.Distance方法。

这种`p.Distance`的表达式叫做选择器，因为他会选择合适的对应p这个对象的Distance方法来执行。选择器也会被用来选择一个struct类型的字段，比如p.X。由于方法和字段都是在同一命名空间，所以如果我们在这里声明一个X方法的话，编译器会报错，因为在调用p.X时会有歧义(译注：这里确实挺奇怪的)。

因为每种类型都有其方法的命名空间，我们在用Distance这个名字的时候，不同的Distance调用指向了不同类型里的Distance方法。让我们来定义一个Path类型，这个Path代表一个线段的集合，并且也给这个Path定义一个叫Distance的方法。